# Extraction

Extraction using the `zipfile` package has its specific features, and this page focuses on describing them.

In [21]:
import seedir as sd
from pathlib import Path
from zipfile import ZipFile
import shutil

We need an archive to use as an example. The following cell creates such an archive in the folder `extraction_files` and shows its file structure. 

In [23]:
files_path = Path("extraction_files")
files_path.mkdir(exist_ok=True)

toy_folder = files_path/"toy_folder"
toy_folder.mkdir(exist_ok=True)

archieve_file_path = (toy_folder/"archived_file")
archieve_file_path.write_text("this is some message")

inner_folder = toy_folder/"inner_folder"
inner_file = inner_folder/"file"
inner_folder.mkdir()
inner_file.write_text("inner file content")

archive_path = files_path/"archive.zip"
with ZipFile(file=archive_path, mode='w') as a:
    a.write(archieve_file_path)
    a.write(inner_file)

sd.seedir(toy_folder)
shutil.rmtree(toy_folder)

toy_folder/
├─inner_folder/
│ └─file
└─archived_file


## Particular file

If you need to extract a particular file, not the whole file tree from an archive, you can use:

- The `extract` method: It's straightforward but copies the whole path in the archive to the filesystem.
- The `open` method for a particular file, combined with binary copying to another stream: This allows more control over the extraction process.

---

The following cell shows using of the `extract` method, and prints the file system of the destimation folder. 

In [24]:
temp_dir = Path("temp_dir")
outer_file_path = temp_dir/"outer_file"
inner_file_path = temp_dir/"inner_file"

with ZipFile(file=archive_path, mode='r') as f:
    f.extract(member=str(archieve_file_path), path=outer_file_path)
    f.extract(member=str(inner_file), path=inner_file_path)

sd.seedir(temp_dir)
shutil.rmtree(temp_dir)

temp_dir/
├─inner_file/
│ └─extraction_files/
│   └─toy_folder/
│     └─inner_folder/
│       └─file
└─outer_file/
  └─extraction_files/
    └─toy_folder/
      └─archived_file


Files were loaded, but their paths are fully copied inside the archive to the destination folder.

The following cell shows the second option - file of archive was opened separately and rewrited to the specified file in filesystem.

In [29]:
dest_file = files_path/"particular_file"

with ZipFile(file=archive_path, mode='r') as zip_archive:
    with (
        zip_archive.open(str(inner_file), mode='r') as source, 
        open(str(dest_file), mode="wb") as destination
    ):
        destination.write(source.read())

sd.seedir(files_path)
print("Content:", dest_file.read_text())
dest_file.unlink()

extraction_files/
├─archive.zip
└─particular_file
Content: inner file content


The file was extracted to the same folder as the archive. In the file tree, there are only the extracted files without any extra folders. Finally, the contents of the extracted file are printed, and they correspond to the originally saved content.